In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import networkx as nx
import asyncio
import aiohttp
import json

In [29]:
import requests
from bs4 import BeautifulSoup

def scrape_episode_data(url, title_text):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the h3 tag with a child span containing the specified text
        title_tag = soup.find('h3', string=title_text)

        if title_tag:
            # Find the corresponding table by searching for the closest table tag after the title
            table = title_tag.find_next('table', class_='box table coloured bordered innerbordered style-basic fill-horiz')

            if table:
                # Initialize lists to store data
                episode_data = []

                # Loop through rows in the table
                for row in table.find_all('tr')[1:]:  # Skip the header row
                    # Extract data from each column in the row
                    columns = row.find_all('td')

                    # Assuming the structure is consistent, extract data from each column
                    episode_number = row.find('th').get_text(strip=True)

                    # Check if anchor tag is present before trying to get its text
                    episode_title_tag = columns[0].find('a')
                    episode_title = episode_title_tag.get_text(strip=True).replace(' ', '_') if episode_title_tag else None

                    # Append data to the list of dictionaries
                    episode_data.append({
                        'Episode Number': episode_number,
                        'Episode Title': episode_title,
                    })

                return episode_data

            else:
                print(f"Table not found after the title with text '{title_text}'.")
                return None

        else:
            print(f"Title with text '{title_text}' not found on the page.")
            return None

    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return None

In [31]:
# Example usage:
url = "https://naruto.fandom.com/wiki/List_of_Animated_Media"
season_titles = ['Naruto: Original', 'Naruto: Shippūden', 'Boruto: Naruto Next Generations']

episodes_seasons_data = {}

for season in season_titles:
    data = scrape_episode_data(url, season)
    if data is not None:
        episodes_seasons_data[season] = data


Title with text 'Naruto: Shippūden' not found on the page.


In [ ]:
# Print the results for debugging
for season, data in episodes_seasons_data.items():
    print(f"\nSeason: {season}")
    if data:
        for episode in data:
            print(episode)
    else:
        print("No data found.")